## DigiSafe
---

In [2]:
:cache 1000
:dep apache-avro
:dep argon2
:dep base64ct = { features = ["alloc"] }
:dep chacha20poly1305
:dep lz4
:dep reed-solomon
:dep serde = { features = ["derive"] }
:dep sha3
:dep getrandom

cache: 1000 MiB


In [3]:
pub mod Database {

    // Database<Unlocked> <-> Avro <-> LZ4 <-> ChaCha20Poly1305 <-> Base64 <-> Database<Locked>
    // Database<Locked> -> Avro -> Reed-Solomon -> Vec<u8> -> name.digisafe

    use serde::{Deserialize, Serialize};
    use std::collections::HashMap;
    use std::io::{Read, Write};

    pub fn hash_lite(msg: &Vec<u8>) -> Vec<u8> {
        use sha3::{Digest, Sha3_256};
        Sha3_256::digest(Sha3_256::digest(msg)).to_vec()
    }

    pub fn hash_heavy(msg: String) -> Vec<u8> {
        use argon2::{Algorithm, Argon2, ParamsBuilder, password_hash::{PasswordHasher, SaltString}, Version};
        let lite_hash = hash_lite(&msg.into());
        let params = ParamsBuilder::new().m_cost(2u32.pow(20)).t_cost(2).p_cost(8).output_len(32).build().unwrap();
        let hasher = Argon2::new(Algorithm::Argon2id, Version::V0x13, params);
        let salt = SaltString::encode_b64(b"digisafe").unwrap();
        let heavy_hash = hasher.hash_password(&lite_hash, &salt).unwrap().hash.unwrap().as_bytes().to_owned();
        hash_lite(&heavy_hash)
    }

    pub fn to_base64(msg: &Vec<u8>) -> String {
        use base64ct::{Base64, Encoding};
        Base64::encode_string(msg)
    }

    pub fn from_base64(msg_enc: &String) -> Vec<u8> {
        use base64ct::{Base64, Encoding};
        Base64::decode_vec(&msg_enc).unwrap()
    }

    pub fn compress(msg: Vec<u8>) -> Vec<u8> {
        use lz4::{Decoder, EncoderBuilder};
        let mut encoder = EncoderBuilder::new().level(9).build(vec![]).unwrap();
        encoder.write(&msg[..]);
        encoder.finish().0
    }

    pub fn decompress(msg_enc: Vec<u8>) -> Vec<u8> {
        use lz4::Decoder;
        let mut msg = vec![];
        {
            let mut decoder = Decoder::new(&msg_enc[..]).unwrap();
            std::io::copy(&mut decoder, &mut msg);
        }
        msg
    }
    
    pub fn encrypt(msg: Vec<u8>, key: Vec<u8>, nonce: Vec<u8>) -> Vec<u8> {
        use chacha20poly1305::{aead::{Aead, KeyInit}, XChaCha20Poly1305};
        let key: [u8; 32] = key.try_into().unwrap();
        let nonce: [u8; 24] = nonce.try_into().unwrap();
        let cipher = XChaCha20Poly1305::new(&key.into());
        cipher.encrypt(&nonce.into(), &msg[..]).unwrap()
    }

    pub fn decrypt(msg_enc: Vec<u8>, key: Vec<u8>, nonce: Vec<u8>) -> Option<Vec<u8>> {
        use chacha20poly1305::{aead::{Aead, KeyInit}, XChaCha20Poly1305};
        let key: [u8; 32] = key.try_into().unwrap();
        let nonce: [u8; 24] = nonce.try_into().unwrap();
        let cipher = XChaCha20Poly1305::new(&key.into());
        if let Ok(msg) = cipher.decrypt(&nonce.into(), msg_enc.as_ref()) {
            Some(msg)
        } else {
            None
        }
    }

    const unlocked_schema_raw: &str = r#"
    {
        "type": "record",
        "name": "unlocked",
        "fields": [
            {"name": "db", "type": {"type": "map", "values": "string"}},
            {"name": "info", "type": {"type": "map", "values": "string"}}
        ]
    }
    "#;

    #[derive(Debug, Deserialize, Serialize)]
    pub struct AvroUnlocked {
        db: HashMap<String, String>,
        info: HashMap<String, String>,
    }
    
    const locked_schema_raw: &str = r#"
    {
        "type": "record",
        "name": "locked",
        "fields": [
            {"name": "db", "type": "string"},
            {"name": "info", "type": {"type": "map", "values": "string"}}
        ]
    }
    "#;

    #[derive(Debug, Deserialize, Serialize)]
    pub struct AvroLocked {
        db: String,
        info: HashMap<String, String>,
    }

    #[derive(Default, Debug)]
    pub struct Unlocked {
        db: HashMap<String, String>,
        info: HashMap<String, String>,
        key: Vec<u8>,
    }

    #[derive(Debug, Default)]
    pub struct Locked {
        db: String,
        info: HashMap<String, String>,
    }


    impl AvroUnlocked {
        pub fn to_unlocked_struct(self, key: Vec<u8>) -> Unlocked {
             Unlocked {
                 db: self.db,
                 info: self.info,
                 key: key,
             }
        }
    }

    impl AvroLocked {
        pub fn to_locked_struct(self) -> Locked {
             Locked {
                 db: self.db,
                 info: self.info,
             }
        }
    }
    
    impl Unlocked {
        
        pub fn new(key: impl Into<String>) -> Self {
            let mut obj = Unlocked {
                db: Default::default(),
                info: Default::default(),
                key: hash_heavy(key.into()),
            };
            let mut randy = [0u8; 32];
            getrandom::getrandom(&mut randy);
            obj.set_info("uid", to_base64(&randy.to_vec()));
            obj
        }
        
        pub fn lock(self) -> Locked {
            let cadb = compress(self.to_avro());
            let nonce = hash_lite(&cadb)[..24].to_vec();
            let mut info = HashMap::<String, String>::new();
            info.extend(self.info);
            info.insert("nonce".into(), to_base64(&nonce));
            let db = to_base64(&encrypt(cadb, self.key, nonce));
            Locked {
                db: db,
                info: info,
            }
        }
    
        pub fn get(&self, akey: &String) -> Option<String> {
            self.db.get(akey).cloned()
        }
    
        
        pub fn set(&mut self, akey: impl Into<String>, aval: impl Into<String>) {
            self.db.insert(akey.into(), aval.into());
        }
    
        pub fn get_info(&self, akey: &String) -> Option<String> {
            self.info.get(akey).cloned()
        }
    
        
        pub fn set_info(&mut self, akey: impl Into<String>, aval: impl Into<String>) {
            self.info.insert(akey.into(), aval.into());
        }

        pub fn merge(&mut self, other: Unlocked) {
            self.db.extend(other.db);
        }

        pub fn to_avro_struct(&self) -> AvroUnlocked {
            AvroUnlocked {
                db: self.db.clone(),
                info: self.info.clone(),
            }
        }

        pub fn to_avro(&self) -> Vec<u8> {
            use apache_avro::{to_value, Schema, Writer};
            let schema = Schema::parse_str(unlocked_schema_raw).unwrap();
            let mut writer = Writer::new(&schema, Vec::new());
            writer.append(to_value(&self.to_avro_struct()).unwrap()).unwrap();
            let avro_dat = writer.into_inner().unwrap();
            avro_dat
        }

        pub fn from_avro(avro_dat: Vec<u8>, key: Vec<u8>) -> Unlocked {
            use apache_avro::{from_value, Reader};
            let reader = Reader::new(&avro_dat[..]).unwrap();
            let db = from_value::<AvroUnlocked>(&reader.last().unwrap().unwrap()).unwrap();
            db.to_unlocked_struct(key)
        }
    }

    /*
    impl std::fmt::Debug for Unlocked {
        fn fmt(&self, objf: &mut std::fmt::Formatter) -> std::fmt::Result {
            write!(objf, "Database::Unlocked: {{ redacted }}")
        }
    }
    */

    impl Locked {
        
        pub fn unlock(self, password: impl Into<String>) -> Option<Unlocked> {
            let key = hash_heavy(password.into());
            let nonce = from_base64(&self.info.get("nonce").unwrap());
            let cradb = decrypt(from_base64(&self.db), key.clone(), nonce);
            if cradb.is_none() {
                return None;
            }
            let radb = decompress(cradb.unwrap());
            let db = Unlocked::from_avro(radb, key);
            Some(db)
        }

        pub fn to_avro_struct(&self) -> AvroLocked {
            AvroLocked {
                db: self.db.clone(),
                info: self.info.clone(),
            }
        }

        pub fn to_avro(&self) -> Vec<u8> {
            use apache_avro::{to_value, Schema, Writer};
            let schema = Schema::parse_str(locked_schema_raw).unwrap();
            let mut writer = Writer::new(&schema, Vec::new());
            writer.append(to_value(&self.to_avro_struct()).unwrap()).unwrap();
            let avro_dat = writer.into_inner().unwrap();
            avro_dat
        }

        pub fn from_avro(avro_dat: Vec<u8>) -> Locked {
            use apache_avro::{from_value, Reader};
            let reader = Reader::new(&avro_dat[..]).unwrap();
            let db = from_value::<AvroLocked>(&reader.last().unwrap().unwrap()).unwrap();
            db.to_locked_struct()
        }


    }

}

In [4]:
let mut db = Database::Unlocked::new("test_key");
db

Unlocked { db: {}, info: {"uid": "qy6rqPCTQbJFBoXD1d6mRzboMa7Gp4owoPVfaZ6HC7s="}, key: [146, 251, 216, 195, 51, 10, 173, 46, 196, 91, 169, 181, 149, 93, 179, 66, 192, 91, 240, 175, 185, 222, 31, 183, 198, 99, 75, 154, 88, 8, 146, 245] }

In [5]:
db.set_info("name", "test_db");
db

Unlocked { db: {}, info: {"name": "test_db", "uid": "qy6rqPCTQbJFBoXD1d6mRzboMa7Gp4owoPVfaZ6HC7s="}, key: [146, 251, 216, 195, 51, 10, 173, 46, 196, 91, 169, 181, 149, 93, 179, 66, 192, 91, 240, 175, 185, 222, 31, 183, 198, 99, 75, 154, 88, 8, 146, 245] }

In [6]:
db.set("k1", "v1");
db.set("k2", "v2");
db.set("k3", "v3");
db

Unlocked { db: {"k3": "v3", "k1": "v1", "k2": "v2"}, info: {"name": "test_db", "uid": "qy6rqPCTQbJFBoXD1d6mRzboMa7Gp4owoPVfaZ6HC7s="}, key: [146, 251, 216, 195, 51, 10, 173, 46, 196, 91, 169, 181, 149, 93, 179, 66, 192, 91, 240, 175, 185, 222, 31, 183, 198, 99, 75, 154, 88, 8, 146, 245] }

In [7]:
let db = db.lock();
db

Locked { db: "5pSYoY0ZKrpOyEWsg096JHgnZLz3yIdDgcCtuSJxS/TOPJpytt9BsxypcoU40LIyb7tm2lFzPqB7orQHB1Npsa45QnzPHdCbR3dIuHfuo9rrAE/NzIHdnjMg58sOXXov+/HmzTnuNRyPpGcyTgPKyzuHsfD9Yvti+ydlUjfoBsEtWFpvRYXr5XINoshDa9+hO8EwV/BcOXU9Z6zZplsO1jl/FJan7qHaGw47xk2VlMQ8bk4XWHkQOaf7dPbMEzjxuV9vFigZFZyX6xAzcWH8Qnuir/5fGAyEnCoTQ2i7N5RD8OQpElYF620JsQKZr8kU4o5nifywPq/JfG/2OcHXD4Nfza0j4CEuSHlS2FcendVecCviDw==", info: {"uid": "qy6rqPCTQbJFBoXD1d6mRzboMa7Gp4owoPVfaZ6HC7s=", "name": "test_db", "nonce": "bS3MhKmHZ8Q0nuqwjzbktlmJ14++svQp"} }

In [8]:
let db = db.unlock("test_key").unwrap();
db

Unlocked { db: {"k1": "v1", "k3": "v3", "k2": "v2"}, info: {"name": "test_db", "uid": "qy6rqPCTQbJFBoXD1d6mRzboMa7Gp4owoPVfaZ6HC7s="}, key: [146, 251, 216, 195, 51, 10, 173, 46, 196, 91, 169, 181, 149, 93, 179, 66, 192, 91, 240, 175, 185, 222, 31, 183, 198, 99, 75, 154, 88, 8, 146, 245] }

In [9]:
let db = db.lock();
db

Locked { db: "RNgBO/pJXJmW5IqIfaM72HA/pxPkL333uxeAOO1WQE6V9hQGTcU6WNi8SQsrlDSMaO7FHrG0CzZzcXxyxnhtS9FAWJvRHAu+83E9wSS6Ck6YZOZn5WyBQmu4VsN1ziAqqSY5ZOfXOld8547FEh0BrzhJo651fAPR5AytJsDxYNLteGWd4QrGk/QaYl5Exi6ceV4q7A44r1IelFDp8ex989gH45ee9NQN33XA+7NZf2TQV72x79NdmWAzzMUbRgdw1e6ZBaScoaZzShGQKWtSfb9LKTveTZPGPmy/Wvyz2bO8eDLW+7us3Gxidxlnjqi/LLS5HNdm99oD2NR3TbxpfL/jfKmo6O1+QsjiHewCvv3C/wUv+/s=", info: {"uid": "qy6rqPCTQbJFBoXD1d6mRzboMa7Gp4owoPVfaZ6HC7s=", "nonce": "73sOQ1R9kvtRCYLulBYl04ssEAJGAXFJ", "name": "test_db"} }

In [10]:
db.to_avro()[..100]

[79, 98, 106, 1, 4, 22, 97, 118, 114, 111, 46, 115, 99, 104, 101, 109, 97, 132, 2, 123, 34, 116, 121, 112, 101, 34, 58, 34, 114, 101, 99, 111, 114, 100, 34, 44, 34, 110, 97, 109, 101, 34, 58, 34, 108, 111, 99, 107, 101, 100, 34, 44, 34, 102, 105, 101, 108, 100, 115, 34, 58, 91, 123, 34, 110, 97, 109, 101, 34, 58, 34, 100, 98, 34, 44, 34, 116, 121, 112, 101, 34, 58, 34, 115, 116, 114, 105, 110, 103, 34, 125, 44, 123, 34, 110, 97, 109, 101, 34, 58]

In [11]:
let db = Database::Locked::from_avro(db.to_avro());
db

Locked { db: "RNgBO/pJXJmW5IqIfaM72HA/pxPkL333uxeAOO1WQE6V9hQGTcU6WNi8SQsrlDSMaO7FHrG0CzZzcXxyxnhtS9FAWJvRHAu+83E9wSS6Ck6YZOZn5WyBQmu4VsN1ziAqqSY5ZOfXOld8547FEh0BrzhJo651fAPR5AytJsDxYNLteGWd4QrGk/QaYl5Exi6ceV4q7A44r1IelFDp8ex989gH45ee9NQN33XA+7NZf2TQV72x79NdmWAzzMUbRgdw1e6ZBaScoaZzShGQKWtSfb9LKTveTZPGPmy/Wvyz2bO8eDLW+7us3Gxidxlnjqi/LLS5HNdm99oD2NR3TbxpfL/jfKmo6O1+QsjiHewCvv3C/wUv+/s=", info: {"nonce": "73sOQ1R9kvtRCYLulBYl04ssEAJGAXFJ", "uid": "qy6rqPCTQbJFBoXD1d6mRzboMa7Gp4owoPVfaZ6HC7s=", "name": "test_db"} }

In [12]:
let db = db.unlock("test_key").unwrap();
db

Unlocked { db: {"k1": "v1", "k2": "v2", "k3": "v3"}, info: {"uid": "qy6rqPCTQbJFBoXD1d6mRzboMa7Gp4owoPVfaZ6HC7s=", "name": "test_db"}, key: [146, 251, 216, 195, 51, 10, 173, 46, 196, 91, 169, 181, 149, 93, 179, 66, 192, 91, 240, 175, 185, 222, 31, 183, 198, 99, 75, 154, 88, 8, 146, 245] }

In [13]:
// todo: Locked -> AvroLocked -> Reed-Solomon -> Vec<u8> -> name.digisafe

In [14]:
// todo: app, nonce, uid, version, revision, name, type, subtype, timestamp

In [15]:
// todo: manage key during locked state. what is current api???